OLD NOTEBOOK: SEE osm_pbf_power_data_extractor.py which does everything.

In [2]:
#check folder

import os, sys, time
(os.path.abspath(''))

'c:\\Users\\Davide\\Git\\pypsa-africa\\data_exploration\\WP5_transmission_assets'

In [3]:
import os, sys, time
#IMPORTANT: RUN SCRIPT FROM THIS SCRIPTS DIRECTORY i.e data_exploration/ TODO: make more robust
## os.chdir(os.path.dirname(os.path.abspath(__file__)))
sys.path.append('../../scripts')
from iso_country_codes import AFRICA_CC



import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
import geoplot
import matplotlib.pyplot as plt
from iso_country_codes import AFRICA_CC
from osm_pbf_power_data_extractor import convert_pd_to_gdf_lines, convert_pd_to_gdf

import logging
logger = logging.getLogger(__name__)

# SUBSTATIONS (Just simple cleaning to test snapping)

### Check old buses

In [4]:
# Load pypsa-eur data
df_all_buses = (pd.read_csv(os.getcwd()+"/entsoegridkit/buses.csv", quotechar="'",
                         true_values='t', false_values='f',
                         dtype=dict(bus_id="str"))
            .set_index("bus_id")
            .drop(['station_id'], axis=1)
            .rename(columns=dict(voltage='v_nom')))

#print(df_all_lines.geometry.unique())
#display(df_all_buses)

# Import data and create final dataframe layout

In [30]:
#----------- SUBSTATIONS -----------
# Load uncleaned data
df_all_substations = gpd.read_file(os.getcwd()+"/data/raw/africa_all_raw_substations.geojson")

# Modification - create final dataframe layout
df_all_substations = df_all_substations.rename(
    columns = {
        "id": "bus_id",
        "tags.voltage": "voltage",
        # "dc", will be added below
        "tags.power": "symbol",
        # "under_construction", will be added below     
        "tags.substation": "tag_substation",
        "Country": "country",  # new/different to PyPSA-Eur
        "Area": "tag_area",
        "lonlat": "geometry",
    }
)

# Add NaN as default
df_all_substations["station_id"] = np.nan
df_all_substations["dc"] = np.nan
df_all_substations["under_construction"] = np.nan
df_all_substations["lon"] = df_all_substations["geometry"].x
df_all_substations["lat"] = df_all_substations["geometry"].y

#Rearrange columns
clist = ["bus_id","station_id","voltage","dc","symbol","under_construction","tag_substation",
         "tag_area","lon", "lat", "geometry","country"]
df_all_substations = df_all_substations[clist]

# make float to integer
df_all_substations["bus_id"] = df_all_substations["bus_id"].astype(int)

#display(df_all_substations)

### Define under_construction, dc, filter "transmission"

In [31]:
df_all_substations["under_construction"] = True
df_all_substations["dc"] = False
df_all_substations = df_all_substations[df_all_substations["tag_substation"] == "transmission"] # keep only rows with indexed "transmission"
#display(df_all_substations)

In [32]:
# df_all_substations.tags_area.count()

In [33]:
# df_all_substations[df_all_substations["tags_substation"] == "transmission"].tags_area.describe()

In [34]:
# df_all_substations[df_all_substations["tags_substation"] == "distribution"].tags_area.describe()

In [35]:
# df_all_substations[df_all_substations["tags_substation"] == "industrial"].tags_area.describe()

In [36]:
# df_all_substations[df_all_substations["tags_substation"].isna()].tags_area.describe()

In [37]:
# df_all_substations["tags_substation"].unique()

### Clean voltage

In [38]:
# Drop any row with Voltage = N/A
df = df_all_substations.dropna(subset=['voltage']) 

#Split semicolon separated cells i.e. [66000;220000] and create new identical rows
lst_col = 'voltage'
x = df.assign(**{lst_col:df[lst_col].str.split(';')})
x = pd.DataFrame({
    col:np.repeat(x[col].values, x[lst_col].str.len())
    for col in x.columns.difference([lst_col])
    }).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]
df_all_substations = x

#display(df_all_substations)

In [39]:
# Create unique bus id's
# The steps below create unique bus id's without loosing the original OSM bus_id 

# Context
# The previous duplication of rows (to split the voltage) lead to a couple of same bus_id

# Method
# Unique bus_id are created by simply adding -1,-2,-3 to the original bus_id
# Every unique id gets a -1 
# If a bus_id exist i.e. three times it it will the counted by cumcount -1,-2,-3 making the id unique

if df_all_substations["bus_id"].count() != df_all_substations["bus_id"].nunique(): # operate only if line_id is not already unique (nunique counts unique values)
    df_all_substations["cumcount"] = df_all_substations.groupby(["bus_id"]).cumcount() # create cumcount column. Cumcount counts 0,1,2,3 the number of duplicates
    df_all_substations["cumcount"] = df_all_substations["cumcount"] + 1 # avoid 0 value for better understanding
    df_all_substations["bus_id"] = df_all_substations["bus_id"].astype(str) + "-" + df_all_substations["cumcount"].values.astype(str) # add cumcount to line_id to make line_id unique
    df_all_substations.drop(columns = "cumcount", inplace=True) # remove cumcount column

#display(df_all_substations)

In [40]:
# Remove all non-numeric values

df_all_substations['voltage'] = df_all_substations['voltage'].apply(lambda x: pd.to_numeric(x, errors='coerce')).astype(float) # if cell can't converted to float -> nan
df_all_substations = df_all_substations.dropna(subset=['voltage']) # Drop any row with Voltage = N/A
df_all_substations.loc[:,"voltage"]  = df_all_substations['voltage'].astype(int)
#df_all_lines['voltage'].unique()

In [41]:
# Keep rows with x > 110 kV as it is considered as transmission level

df_all_substations = df_all_substations[df_all_substations.voltage > 110000]

In [42]:
# display(df_all_substations)
# display(df_all_substations['voltage'].unique())

In [43]:
## Generate Files (CSV+GeoJSON) 

#### CSV
outputfile_partial = os.path.join(os.getcwd(), "data", "clean", "africa_all" + "_buses" + "_clean") # Output file directory

if not os.path.exists(outputfile_partial):
    os.makedirs(os.path.dirname(outputfile_partial), exist_ok=True) #  create clean directoryif not already exist
    
df_all_substations.to_csv(outputfile_partial + ".csv")  # Generate CSV


#### GEOJSON

df_all_substations = gpd.GeoDataFrame(df_all_substations, geometry="geometry",crs="EPSG:4326")
df_all_substations.to_file(outputfile_partial + ".geojson", driver="GeoJSON")    


ERROR:fiona._env:c:\Users\Davide\Git\pypsa-africa\data_exploration\WP5_transmission_assets\data\clean\africa_all_buses_clean.geojson: No such file or directory


# LINES 

### Check old unique values

In [44]:
# Load pypsa-eur data
df_all_lines = (pd.read_csv(os.getcwd()+"/entsoegridkit/lines.csv", quotechar="'", true_values='t', false_values='f',
                         dtype=dict(line_id='str', bus0='str', bus1='str',
                                    underground="bool", under_construction="bool")).set_index('line_id').rename(columns=dict(voltage='v_nom', circuits='num_parallel')))

#print(df_all_lines.geometry.unique())
#display(df_all_lines)

# Import data and create final dataframe layout

### Load cables

In [45]:
# Load raw cable data
df_cables = gpd.read_file(os.getcwd()+"/data/raw/africa_all_raw_cables.geojson") 

# Modification - create final dataframe layout
df_cables = df_cables.rename(
    columns = {
        "id": "line_id",
        "tags.voltage": "voltage",
        "tags.circuits": "circuits",
        "tags.cables": "cables",
        "tags.frequency": "tag_frequency",
        "tags.power": "tag_type",
        "tags.location": "tag_location",
        "lonlat": "geometry",
        "Country": "country",  # new/different to PyPSA-Eur
        "Length": "length",
    }
)

# Add NaN as default
df_cables["bus0"] = np.nan
df_cables["bus1"] = np.nan
#df_all_cables["length"] = np.nan # Now in dataset
df_cables["underground"] = np.nan
df_cables["under_construction"] = np.nan

#Rearrange columns
clist = ["line_id","bus0","bus1","voltage","circuits","length","underground",
         "under_construction","tag_type","tag_frequency", "tag_location","geometry", "country"]
df_cables = df_cables[clist]

# make float to integer
df_cables["line_id"] = df_cables["line_id"].astype(int)


#display(df_cables)
#df_all_cables[df_all_cables['tag_location']== "overground"]
#df_all_cables["tags.location"].unique()

### Load lines

In [46]:
# Load raw line data
df_lines = gpd.read_file(os.getcwd()+"/data/raw/africa_all_raw_lines.geojson") 

# Modification - create final dataframe layout
df_lines = df_lines.rename(
    columns = {
        "id": "line_id",
        "tags.voltage": "voltage",
        "tags.circuits": "circuits",
        "tags.cables": "cables",
        "tags.frequency": "tag_frequency",
        "tags.power": "tag_type",
        "lonlat": "geometry",
        "Country": "country",  # new/different to PyPSA-Eur
        "Length": "length",
    }
)

# Add NaN as default
df_lines["bus0"] = np.nan
df_lines["bus1"] = np.nan
#df_all_lines["length"] = np.nan # commented because, we have now length data
df_lines["underground"] = np.nan
df_lines["under_construction"] = np.nan

#Rearrange columns
clist = ["line_id","bus0","bus1","voltage","circuits","length","underground",
         "under_construction","tag_type","tag_frequency", "cables","geometry", "country"]
df_lines = df_lines[clist]

#display(df_all_lines)

### Combine cable and line to one  "df_all_lines" dataset

In [47]:
df_all_lines = pd.concat([df_lines,df_cables])
# df_all_lines

### Define underground, under_construction information, frequency, circuits

In [48]:
# under construction
df_all_lines["under_construction"] = False # default. Not more information atm available

# underground
df_all_lines["underground"] = (df_all_lines["tag_type"] == "cable") # Simplified. If tag_type cable then underground is True. 
# More information extractable for "underground" by looking at "tag_location".
if 'tag_location' in df_all_lines: # drop column if exist
    df_all_lines.drop(columns = "tag_location", inplace=True)

# frequency
df_all_lines["tag_frequency"] = 50
#df_all_lines["tag_frequency"].unique()

# circuits
if df_all_lines["cables"].dtype != int: # if not int make int
    df_all_lines.loc[(df_all_lines["cables"] < "3") | df_all_lines["cables"].isna(), "cables"] = "0" #HERE. "0" if cables "None", "nan" or "1"
    df_all_lines["cables"] = df_all_lines["cables"].astype("int")
if 4 or 5 in df_all_lines["cables"].values: # downgrade 4 and 5 cables to 3... 
    # Reason: 4 cables have 1 lighting protection cables, 5 cables has 2 LP cables - not transferring energy; 
    # see https://hackaday.com/2019/06/11/a-field-guide-to-transmission-lines/
    df_all_lines.loc[(df_all_lines["cables"] == 4) | (df_all_lines["cables"] == 5), "cables"] = 3 # where circuits are "0" make "1"
df_all_lines.loc[df_all_lines["circuits"].isna(), "circuits"] = df_all_lines.loc[df_all_lines['circuits'].isna(), "cables"] / 3 # one circuit contains 3 cables
df_all_lines["circuits"] = df_all_lines["circuits"].astype(int)
df_all_lines.loc[(df_all_lines["circuits"] == "0") | (df_all_lines["circuits"] == 0), "circuits"] = 1 # where circuits are "0" make "1"

if 'cables' in df_all_lines: # drop column if exist
    df_all_lines.drop(columns = "cables", inplace=True)

# df_all_lines["circuits"].unique()
# df_all_lines["cables"].unique()
# display(df_all_lines)

### Clean voltage

In [49]:
# Drop any row with Voltage = N/A
df = df_all_lines.dropna(subset=['voltage']) 

#Split semicolon separated cells i.e. [66000;220000] and create new identical rows
lst_col = 'voltage'
x = df.assign(**{lst_col:df[lst_col].str.split(';')})
x = pd.DataFrame({
    col:np.repeat(x[col].values, x[lst_col].str.len())
    for col in x.columns.difference([lst_col])
    }).assign(**{lst_col:np.concatenate(x[lst_col].values)})[x.columns.tolist()]
df_all_lines = x

#display(df_all_lines)

In [50]:
# Create unique line_id's
# The steps below create unique line_id's without loosing the original OSM line_id 

# Context
# The previous duplication of rows (to split the voltage) lead to a couple of same line_id (about 30% of dataset)

# Method
# Unique line_id are created by simply adding -1,-2,-3 to the original line_id
# Every unique id gets a -1 
# If a line_id exist i.e. three times it it will the counted by cumcount -1,-2,-3 making the id unique

if df_all_lines["line_id"].count() != df_all_lines["line_id"].nunique(): # operate only if line_id is not already unique (nunique counts unique values)
    df_all_lines["cumcount"] = df_all_lines.groupby(["line_id"]).cumcount() # create cumcount column. Cumcount counts 0,1,2,3 the number of duplicates
    df_all_lines["cumcount"] = df_all_lines["cumcount"] + 1 # avoid 0 value for better understanding
    df_all_lines["line_id"] = df_all_lines["line_id"].astype(str) + "-" + df_all_lines["cumcount"].values.astype(str) # add cumcount to line_id to make line_id unique
    df_all_lines.drop(columns = "cumcount", inplace=True) # remove cumcount column

#display(df_all_lines)

In [51]:
# Remove all non-numeric values

df_all_lines.loc[:,"voltage"] = df_all_lines['voltage'].apply(lambda x: pd.to_numeric(x, errors='coerce')).astype(float) # if cell can't converted to float -> nan
df_all_lines = df_all_lines.dropna(subset=['voltage']) # Drop any row with Voltage = N/A
df_all_lines.loc[:,"voltage"]  = df_all_lines['voltage'].astype(int)
#df_all_lines['voltage'].unique()

C:\Users\Davide\.conda\envs\main_env\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [52]:
# Keep rows with x > 110 kV as it is considered as transmission level
df_all_lines = df_all_lines[df_all_lines.voltage > 110000]

# Remove lines that are shorter than 100m
#df_all_lines = df_all_lines[df_all_lines.length > 100]

In [53]:
# display(df_all_lines)
# display(df_all_lines['voltage'].unique())
# display(df_all_lines['length'].describe())

In [54]:
## Generate Files (CSV+GeoJSON) 


### CSV
outputfile_partial = os.path.join(os.getcwd(), "data", "clean", "africa_all" + "_lines" + "_clean") # Output file directory

if not os.path.exists(outputfile_partial):
    os.makedirs(os.path.dirname(outputfile_partial), exist_ok=True) #  create clean directoryif not already exist
    
df_all_lines.to_csv(outputfile_partial + ".csv")  # Generate CSV


### GEOJSON
df_all_lines = gpd.GeoDataFrame(df_all_lines, geometry="geometry",crs="EPSG:4326")
df_all_lines.to_file(outputfile_partial + ".geojson", driver="GeoJSON")    
  


ERROR:fiona._env:c:\Users\Davide\Git\pypsa-africa\data_exploration\WP5_transmission_assets\data\clean\africa_all_lines_clean.geojson: No such file or directory


# GENERATORS

In [58]:
# ----------- Generator -----------

#Load uncleaned data
df_all_generators = pd.read_csv(os.getcwd()+"/data/raw/africa_all_raw_generators.csv")

#Clean data
df_all_generators = df_all_generators.reset_index(drop=True)
df_all_generators = df_all_generators[df_all_generators['tags.generator:output:electricity'].astype(str).str.contains('MW')] #removes boolean 
df_all_generators['tags.generator:output:electricity'] = df_all_generators['tags.generator:output:electricity'].str.extract('(\d+)').astype(float)
df_all_generators = df_all_generators.rename(columns = {'tags.generator:output:electricity':"power_output_MW"})


## Generate Files

#CSV
outputfile_partial = os.path.join(os.getcwd(),'data','africa_all'+'_generators'+'_cleaned.')
df_all_generators.to_csv(outputfile_partial + 'csv') # Generate CSV

#GeoJSON
# gdf_generators = convert_pd_to_gdf(df_all_generators)
# gdf_generators.to_file(outputfile_partial+'geojson', driver="GeoJSON")  # Generate GeoJson


#display(df_all_generators)

AttributeError: 'DataFrame' object has no attribute 'to_file'